In [75]:
import sys
from nltk.stem import *
import nltk
import json
from pattern.en import singularize
import argparse
import tqdm
import time
import random
lemma = nltk.wordnet.WordNetLemmatizer()

def combine_coco_captions(annotation_path):

    # if not os.path.exists('%s/captions_%s2014.json' %(annotation_path, 'val')):
    #     raise Exception("Please download MSCOCO caption annotations for val set")
    # if not os.path.exists('%s/captions_%s2014.json' %(annotation_path, 'train')):
    #     raise Exception("Please download MSCOCO caption annotations for train set")
    with open(annotation_path) as file:
        all_caps = json.load(file)
    return all_caps 


def combine_coco_instances(annotation_path):

    # if not os.path.exists('%s/instances_%s2014.json' %(annotation_path, 'val')):
    #     raise Exception("Please download MSCOCO instance annotations for val set")
    # if not os.path.exists('%s/instances_%s2014.json' %(annotation_path, 'train')):
    #     raise Exception("Please download MSCOCO instance annotations for train set")

    # val_instances = json.load(open('%s/instances_%s2014.json' %(annotation_path, 'val')))
    # train_instances = json.load(open('%s/instances_%s2014.json' %(annotation_path, 'train')))
    # all_instances = {'info': train_instances['info'],
    #                  'licenses': train_instances['licenses'],
    #                  'type': train_instances['licenses'],
    #                  'categories': train_instances['categories'],
    #                  'images': train_instances['images'] + val_instances['images'],
    #                  'annotations': val_instances['annotations'] + train_instances['annotations']}
    # train_instances.extend(val_instances)
    # all_instances = train_instances
    with open(annotation_path) as file:
        all_instances = json.load(file)
    return all_instances 

class CHAIR(object):

    def __init__(self, imids):

        self.imid_to_objects = {imid: [] for imid in imids}

        #read in synonyms
        # synonym_path = ""
        synonyms = open('C:/Users/hutter_sadan/Desktop/UCAS/blip2/data/synonyms.txt').readlines()
        synonyms = [s.strip().split(', ') for s in synonyms]
        self.synomyms = {}
        for s_line_ls in synonyms:
            for single_s in s_line_ls:
                self.synomyms[single_s] = s_line_ls[0]
        print(self.synomyms)
        self.mscoco_objects = [] #mscoco objects and *all* synonyms
        self.inverse_synonym_dict = {}
        
        # for synonym in synonyms:
        #     self.mscoco_objects.extend(synonym) #["all names"]
        #     for s in synonym:
        #         self.inverse_synonym_dict[s] = synonym[0]

        #read in synonyms
        synonym_path = "C:/Users/hutter_sadan/Desktop/UCAS/blip2/data/object_synsets.json"
        with open(synonym_path) as file:
            synonyms = json.load(file)
        for object_key,object_value in synonyms.items():
            self.mscoco_objects.append(object_key)
            self.inverse_synonym_dict[object_key] = object_value.split('.')[0]
            #"black hole": "hole.n.01"

        #Some hard coded rules for implementing CHAIR metrics on MSCOCO
        
        # #common 'double words' in MSCOCO that should be treated as a single word
        # coco_double_words = ['motor bike', 'motor cycle', 'air plane', 'traffic light', 'street light', 'traffic signal', 'stop light', 'fire hydrant', 'stop sign', 'parking meter', 'suit case', 'sports ball', 'baseball bat', 'baseball glove', 'tennis racket', 'wine glass', 'hot dog', 'cell phone', 'mobile phone', 'teddy bear', 'hair drier', 'potted plant', 'bow tie', 'laptop computer', 'stove top oven', 'hot dog', 'teddy bear', 'home plate', 'train track']
        
        # #Hard code some rules for special cases in MSCOCO
        # #qualifiers like 'baby' or 'adult' animal will lead to a false fire for the MSCOCO object 'person'.  'baby bird' --> 'bird'.
        # animal_words = ['bird', 'cat', 'dog', 'horse', 'sheep', 'cow', 'elephant', 'bear', 'zebra', 'giraffe', 'animal', 'cub']
        # #qualifiers like 'passenger' vehicle will lead to a false fire for the MSCOCO object 'person'.  'passenger jet' --> 'jet'.
        # vehicle_words = ['jet', 'train']
        
        # #double_word_dict will map double words to the word they should be treated as in our analysis
        
        # self.double_word_dict = {}
        # for double_word in coco_double_words:
        #     self.double_word_dict[double_word] = double_word
        # for animal_word in animal_words:
        #     self.double_word_dict['baby %s' %animal_word] = animal_word
        #     self.double_word_dict['adult %s' %animal_word] = animal_word
        # for vehicle_word in vehicle_words:
        #     self.double_word_dict['passenger %s' %vehicle_word] = vehicle_word
        # self.double_word_dict['bow tie'] = 'tie'
        # self.double_word_dict['toilet seat'] = 'toilet'
        # self.double_word_dict['wine glas'] = 'wine glass'

    def _load_generated_captions_into_evaluator(self, cap_file):

        '''
        Meant to save time so imid_to_objects does not always need to be recomputed.
        '''
        #Read in captions        
        self.caps, imids = load_generated_captions(cap_file)

        assert imids == set(self.imid_to_objects.keys())

    def caption_to_words(self, caption):
    
        '''
        Input: caption
        Output: MSCOCO words in the caption
        '''
    
        #standard preprocessing
        if caption == None:
            return [],[],[],[]
        words = nltk.word_tokenize(caption.lower())
        words = [singularize(w) for w in words]
        words = list(set(words))
        raw_words = words
        #replace double words
        #i = 0
        # double_words = []
        # idxs = []
        # while i < len(words):
        #    idxs.append(i) 
        #    double_word = ' '.join(words[i:i+2])
        #    if double_word in self.double_word_dict: 
        #        double_words.append(self.double_word_dict[double_word])
        #        i += 2
        #    else:
        #        double_words.append(words[i])
        #        i += 1
        # words = double_words

        
        #toilet seat is not chair (sentences like "the seat of the toilet" will fire for "chair" if we do not include this line)
        #if ('toilet' in words) & ('seat' in words): words = [word for word in words if word != 'seat']
    
        #get synonyms for all words in the caption
        idxs = [i for i in range(len(words))]
        idxs = [idxs[idx] for idx, word in enumerate(words) \
                if word in set(self.mscoco_objects)]
        words = [word for word in words if word in set(self.mscoco_objects)]
        node_words = []
        for word in words:
            node_words.append(self.inverse_synonym_dict[word])
        #return all the MSCOCO objects in the caption
        return words, node_words, idxs,raw_words

    def get_annotations_from_segments(self,object_json_path):
        '''
        Add objects taken from MSCOCO segmentation masks
        '''

        # coco_segments = combine_coco_instances(self.coco_path)
        # segment_annotations = coco_segments['annotations']

        # #make dict linking object name to ids
        # id_to_name = {} #dict with id to synsets 
        # for cat in coco_segments['categories']:
        #     id_to_name[cat['id']] = cat['name']
        # #类别的id和类别的name之间的对应
        
        # for i, annotation in enumerate(segment_annotations):
        #     sys.stdout.write("\rGetting annotations for %d/%d segmentation masks" 
        #                       %(i, len(segment_annotations)))
        #     imid = annotation['image_id']
        #     if imid in self.imid_to_objects:
        #         node_word = self.inverse_synonym_dict[id_to_name[annotation['category_id']]]
        #         self.imid_to_objects[imid].append(node_word)
        # print ("\n")
        # for imid in self.imid_to_objects:
        #     self.imid_to_objects[imid] = set(self.imid_to_objects[imid])


        # #return self.imid_to_objects 
        # #img_id:node_word

        coco_segment = combine_coco_instances(object_json_path) #list
        #import ipdb;ipdb.set_trace()
        for i in tqdm.tqdm(range(len(coco_segment)),colour = 'blue'):
            temp_object = coco_segment[i] # dict
            img_id = temp_object['image_id']
            object_segment = temp_object['objects'] #list  length == number of object
            if img_id in self.imid_to_objects:
                for j in range(len(object_segment)):
                    object_synsets_name = object_segment[j]['synsets']
                    if object_synsets_name != []:
                        object_synsets_name = object_synsets_name[0] #list[-1]
                        object_synsets_name = object_synsets_name.split('.')[0]
                        if object_synsets_name in self.synomyms:
                            object_synsets_name = self.synomyms[object_synsets_name]
                        self.imid_to_objects[img_id].append(object_synsets_name)
        
        for img_id in self.imid_to_objects:
            self.imid_to_objects[img_id] = set(self.imid_to_objects[img_id])
        #import ipdb;ipdb.set_trace()
    
    def get_annotations_from_captions(self,question_answer_path):
        '''
        Add objects taken from MSCOCO ground truth captions 
        '''

        # coco_caps = combine_coco_captions(question_answer_path)
        # caption_annotations = coco_caps['annotations']

        # for i, annotation in enumerate(caption_annotations):
        #     sys.stdout.write('\rGetting annotations for %d/%d ground truth captions' 
        #                       %(i, len(coco_caps['annotations'])))
        #     imid = annotation['image_id']
        #     if imid in self.imid_to_objects:
        #         _, node_words, _, _ = self.caption_to_words(annotation['caption'])
        #         self.imid_to_objects[imid].update(node_words)
        # print ("\n")

        # for imid in self.imid_to_objects:
        #     self.imid_to_objects[imid] = set(self.imid_to_objects[imid])

        #return self.imid_to_objects
        #img_id node_word
        
        coco_caps = combine_coco_captions(question_answer_path)
        for i in tqdm.tqdm(range(len(coco_caps)),colour = 'green'):
            temp_question = coco_caps[i]#dict
            for j in range(len(temp_question['qas'])):
                img_id = temp_question['qas'][j]['image_id']
                if img_id in self.imid_to_objects:
                    single_caption = temp_question['qas'][j]['question']
                    words, node_words, _,_=self.caption_to_words(single_caption)
                    for node_word in node_words:
                        # self.imid_to_objects[img_id] 是一个set
                        if node_word in self.synomyms:
                            node_word = self.synomyms[node_word]
                        if node_word == "g":
                            print(node_word)
                            print(node_words)
                        #print(node_word)            
                        self.imid_to_objects[img_id].update(node_word)
        for image_id in self.imid_to_objects:
            self.imid_to_objects[image_id] = set(self.imid_to_objects[image_id])
            #print(self.imid_to_objects)
        #import ipdb;ipdb.set_trace()

    def get_annotations(self):

        '''
        Get annotations from both segmentation and captions.  Need both annotation types for CHAIR metric.
        '''

        self.get_annotations_from_segments("C:/Users/hutter_sadan/Desktop/UCAS/blip2/data/objects.json") 
        self.get_annotations_from_captions("C:/Users/hutter_sadan/Desktop/UCAS/blip2/data/question_answers.json") 

    def compute_chair(self, cap_file):
    
        '''
        Given ground truth objects and generated captions, determine which sentences have hallucinated words.
        '''
    
        self._load_generated_captions_into_evaluator(cap_file) #generate_caption_file

        imid_to_objects = self.imid_to_objects
        caps = self.caps #caps is a list
 
        num_caps = 0.
        num_hallucinated_caps = 0.
        hallucinated_word_count = 0.
        coco_word_count = 0.

        output = {'sentences': []} 
    
        for i in tqdm.tqdm(range(len(caps[:10])),colour = 'red'):
            cap_eval = caps[i]
            cap = cap_eval['caption']
            imid = cap_eval['image_id']

            #get all words in the caption, as well as corresponding node word
            words, node_words, idxs,raw_words = self.caption_to_words(cap) #single caption 
            gt_objects = imid_to_objects[imid] #[node words]
            cap_dict = {'image_id': cap_eval['image_id'], 
                        'caption': cap,
                        'mscoco_hallucinated_words': [],
                        'mscoco_gt_words': list(gt_objects),
                        'mscoco_generated_words': list(node_words),
                        'hallucination_idxs': [], 
                        'words': raw_words 
                        }
   
            cap_dict['metrics'] = {'Bleu_1': 0,
                                   'Bleu_2': 0,
                                   'Bleu_3': 0,
                                   'Bleu_4': 0,
                                   'METEOR': 0,
                                   'CIDEr': 0,
                                   'SPICE': 0,
                                   'ROUGE_L': 0,
                                   'CHAIRs': 0,
                                   'CHAIRi': 0}
 
            #count hallucinated words
            coco_word_count += len(node_words) #生成的caption里面存在的object个数。 
            hallucinated = False
            print('gt_objects:{}'.format(gt_objects))
            #print("words:{}".format(words))
            for word, node_word, idx in zip(words, node_words, idxs): #words是一个list
                if node_word in self.synomyms:
                    node_word = self.synomyms[node_word]
                if node_word in gt_objects:
                    print("true node_word:{}".format(node_word))
                    pass
                if node_word not in gt_objects:
                    print("negetive node_word:{}".format(node_word))
                    hallucinated_word_count += 1 
                    cap_dict['mscoco_hallucinated_words'].append((word, node_word))
                    cap_dict['hallucination_idxs'].append(idx)
                    hallucinated = True      
    
            #count hallucinated caps
            num_caps += 1
            if hallucinated:
               num_hallucinated_caps += 1
    
            cap_dict['metrics']['CHAIRs'] = int(hallucinated)
            cap_dict['metrics']['CHAIRi'] = 0.
            if len(words) > 0:
                cap_dict['metrics']['CHAIRi'] = len(cap_dict['mscoco_hallucinated_words'])/float(len(words))
            #time.sleep(2)
            output['sentences'].append(cap_dict)
 
        chair_s = (num_hallucinated_caps/num_caps)
        chair_i = (hallucinated_word_count/coco_word_count)
    
        output['overall_metrics'] = {'Bleu_1': 0,
                                     'Bleu_2': 0,
                                     'Bleu_3': 0,
                                     'Bleu_4': 0,
                                     'METEOR': 0,
                                     'CIDEr': 0,
                                     'SPICE': 0,
                                     'ROUGE_L': 0,
                                     'CHAIRs': chair_s,
                                     'CHAIRi': chair_i}
    
        return output 

def load_generated_captions(cap_file,top_beam=True):
    #Read in captions        
    #cap_file [{'caption':,'image_id':},{}]
    
    #caps_data = json.load(open(cap_file))
    #caps = caps_data['caption_image_id']
    #imids = set([cap['image_id'] for cap in caps])
    #
    #    try:
    #        caps = caps['imgToEval'].values()
    #        imids = set([cap['image_id'] for cap in caps])
    #    except:
    #        raise Exception("Expect caption file to consist of a dectionary with sentences correspdonding to the key 'imgToEval'")
    
    single_qustion_dict = json.load(open(cap_file))
    image_name_ls = []
    image_value_ls = []
    count = 0
    for image_name,image_value in single_qustion_dict.items():
        image_name_ls.append(image_name)
        image_value_ls.append(image_value)
    caps = []
    for idx in tqdm.tqdm(range(len(image_value_ls[:10]))):
        
        image_name = image_name_ls[idx]
        image_value = image_value_ls[idx]
        if type(image_value) == list:
            continue
        top_k_list = image_value["top_k_text"]
        beam_list = image_value['beam_text:']
        my_list = [0, 1, 2, 3]
        random_element = random.choice(my_list)
        temp_idx = -1
        temp_top_k_len = 128
        if top_k_list == None or beam_list == None:
            continue
        for i in range(len(top_k_list)):
            if len(top_k_list[i]) < temp_top_k_len:
                temp_idx = i
                temp_top_k_len = len(top_k_list[i])
        
        rs_top_k = top_k_list[temp_idx] if temp_idx != -1 else None
        #rs_top_k = top_k_list[random_element]
        temp_idx = -1
        temp_beam_len = 128 
        for i in range(len(beam_list)):
            if len(beam_list[i]) < temp_beam_len:
                temp_idx = i
                temp_beam_len = len(beam_list[i])
        my_list = [0, 1]
        random_element = random.choice(my_list)
        rs_beam = beam_list[temp_idx] if temp_idx != -1 else None
        #rs_beam = beam_list[random_element]
        if top_beam:
            caps.append({'caption':rs_top_k,'image_id':int(image_name[:-4])})
        else:
            caps.append({'caption':rs_beam,'image_id':int(image_name[:-4])})
        #caps.append({'caption':rs_top_k,'image_id':int(image_name[:-4])})
        #caps.append({'caption':rs_beam,'image_id':int(image_name[:-4])})
    imids = set([cap['image_id'] for cap in caps])
    return caps, imids

def save_hallucinated_words(cap_file, cap_dict): 
    tag = cap_file.split('/')[-1] 
    with open('C:/Users/hutter_sadan/Desktop/UCAS/blip2/output/hallucination/hallucinated_words_%s' %tag, 'a+') as f:
        json.dump(cap_dict, f)

def print_metrics(hallucination_cap_dict, quiet=False):
    sentence_metrics = hallucination_cap_dict['overall_metrics']
    metric_string = "%0.01f\t%0.01f\t%0.01f\t%0.01f\t%0.01f" %(
                                                  sentence_metrics['SPICE']*100,
                                                  sentence_metrics['METEOR']*100,
                                                  sentence_metrics['CIDEr']*100,
                                                  sentence_metrics['CHAIRs']*100,
                                                  sentence_metrics['CHAIRi']*100)

    if not quiet:
        print ("SPICE\tMETEOR\tCIDEr\tCHAIRs\tCHAIRi")
        print (metric_string)

    else:
        return metric_string
 

In [76]:
cap_file = "C:/Users/hutter_sadan/Desktop/UCAS/blip2/data/test_total_VG_BLIP2_100.json"
_, imids = load_generated_captions(cap_file,True)



100%|██████████| 10/10 [00:00<?, ?it/s]


In [77]:
evaluator = CHAIR(imids) 


{'person': 'person', 'girl': 'person', 'boy': 'person', 'man': 'person', 'woman': 'person', 'kid': 'person', 'child': 'person', 'chef': 'person', 'baker': 'person', 'people': 'person', 'adult': 'person', 'rider': 'person', 'children': 'person', 'baby': 'person', 'worker': 'person', 'passenger': 'person', 'sister': 'person', 'biker': 'person', 'policeman': 'person', 'cop': 'person', 'officer': 'person', 'lady': 'person', 'cowboy': 'person', 'bride': 'person', 'groom': 'person', 'male': 'person', 'female': 'person', 'guy': 'person', 'traveler': 'person', 'mother': 'person', 'father': 'person', 'gentleman': 'person', 'pitcher': 'person', 'player': 'person', 'skier': 'person', 'snowboarder': 'person', 'skater': 'person', 'skateboarder': 'person', 'foreigner': 'person', 'caller': 'person', 'offender': 'person', 'coworker': 'person', 'trespasser': 'person', 'patient': 'person', 'politician': 'person', 'soldier': 'person', 'grandchild': 'person', 'serviceman': 'person', 'walker': 'person', 'd

In [78]:
evaluator.get_annotations()


  0%|          | 0/108077 [00:00<?, ?it/s]

lamp
flag
street
clock
object
picture
face
clock
uracil


 10%|█         | 11141/108077 [00:00<00:03, 25876.56it/s]

clock
hand
window
range_pole
range_pole
clock_time
picture


 34%|███▎      | 36408/108077 [00:00<00:01, 49572.06it/s]

...

color
sky
photograph
color
pole
light
person
blackboard
symbol


 88%|████████▊ | 94604/108077 [00:03<00:00, 26474.34it/s]

picture
disability
bathroom
toilet
color
wall
person
blackboard
use
bathroom
rubbish


100%|██████████| 108077/108077 [00:03<00:00, 30222.36it/s]


type
food
food
photograph
{2384832: {'shoulder_pad', 'fender', 's', 'dirt_track', 'goggles', 'v', 'l', 'r', 'brake', 'mouthpiece', 'c', 'gummed_label', 'u', 't', 'grass', 'logo', 'e', 'uniform', 'b', 'knee_pad', 'd', 'glove', 'soil', 'tire', 'h', 'object', 'boot', 'm', 'o', 'i', 'person', 'a', 'p', 'helmet', 'y', 'vest', 'n'}, 2362946: {'part', 's', 'photograph', 'umbrella', 'floor', 'l', 'g', 'r', 'c', 'u', 't', 'grass', 'e', 'road', 'd', 'h', 'line', 'light', 'n', 'edge', 'corner', 'surface', 'o', 'person', 'a', 'p', 'name'}, 2339879: {'sidewalk', 'tree', 'f', 'cap', 's', 'door', 'air_conditioner', 'doorway', 'r', '_', 'u', 'c', 'chain', 'fireplug', 't', 'e', 'building', 'numeral', 'w', 'pole', 'window', 'light', 'm', 'street', 'glass', 'pavement', 'a', 'screen', 'n'}, 2370057: {'poster', 'jean', 's', 'couch', 'shoe', 'photograph', 'l', 'g', 'wristband', 'shirt', 'r', 'hand', 'c', 'u', '_', 't', 'grass', 'e', 'seating', 'b', 'd', 'j', 'ring', 'face', 'land', 'h', 'foot', 'trouser', '

In [61]:
cap_dict = evaluator.compute_chair(cap_file) 
print_metrics(cap_dict)

  2%|▏         | 5/300 [00:00<00:07, 41.44it/s]

gt_objects:{'aftermath', 'w', 'door', 'land', 'tire', 'h', 'pole', 'side', 'water', 'boat', 'l', 'g', 'o', 'r', 'barrier', 'i', 'c', 'u', 'covering', 'a', 'p', 't', 'e', 'buoy', 'screen', 'n', 'b'}
true node_word:boat
gt_objects:{'aftermath', 'w', 'door', 'land', 'tire', 'h', 'pole', 'side', 'water', 'boat', 'l', 'g', 'o', 'r', 'barrier', 'i', 'c', 'u', 'covering', 'a', 'p', 't', 'e', 'buoy', 'screen', 'n', 'b'}
true node_word:boat
gt_objects:{'aftermath', 'w', 'door', 'land', 'tire', 'h', 'pole', 'side', 'water', 'boat', 'l', 'g', 'o', 'r', 'barrier', 'i', 'c', 'u', 'covering', 'a', 'p', 't', 'e', 'buoy', 'screen', 'n', 'b'}
true node_word:water
true node_word:boat
gt_objects:{'poster', 'jean', 's', 'couch', 'shoe', 'photograph', 'l', 'g', 'wristband', 'shirt', 'r', 'hand', 'c', 'u', '_', 't', 'grass', 'e', 'seating', 'b', 'd', 'j', 'ring', 'face', 'land', 'h', 'foot', 'trouser', 'remote_control', 'stick', 'm', 'o', 'hair', 'k', 'i', 'person', 'smile', 'shelf', 'p', 'a', 'item', 'n'}


  6%|▌         | 17/300 [00:00<00:06, 43.82it/s]

gt_objects:{'railing', 's', 'chair', 'l', 'writing', 'r', 'c', 'u', 't', 'e', 'b', 'd', 'blackboard', 'w', 'eraser', 'toilet', 'h', 'bowl', 'window', 'eyelid', 'toilet_tissue', 'm', 'o', 'ledge', 'wall', 'k', 'frame', 'board', 'i', 'a', 'p', 'y', 'holder', 'n', 'shadow'}
negetive node_word:sink
true node_word:toilet
true node_word:blackboard
negetive node_word:bathroom
gt_objects:{'d', 'cup', 'f', 'plate', 'h', 'hotdog', 'catsup', 'eyelid', 'newspaper', 'dog', 'g', 'o', 'letter', 'y', 'r', 'condiment', 'helium', 'paper_plate', 'a', 'napkin', 'p', 't', 'e', 'word', 'text', 'dining table'}
true node_word:dog
gt_objects:{'d', 'cup', 'f', 'plate', 'h', 'hotdog', 'catsup', 'eyelid', 'newspaper', 'dog', 'g', 'o', 'letter', 'y', 'r', 'condiment', 'helium', 'paper_plate', 'a', 'napkin', 'p', 't', 'e', 'word', 'text', 'dining table'}
true node_word:dog
gt_objects:{'d', 'cup', 'f', 'plate', 'h', 'hotdog', 'catsup', 'eyelid', 'newspaper', 'dog', 'g', 'o', 'letter', 'y', 'r', 'condiment', 'helium'

 10%|▉         | 29/300 [00:00<00:05, 52.58it/s]

...

gt_objects:{'branch', 'tree', 's', 'apparel', 'sky', 'l', 'g', 'shirt', 'r', '_', 'u', 'c', 't', 'e', 'building', 'jacket', 'field', 'b', 'd', 'string', 'awning', 'w', 'land', 'h', 'window', 'background', 'o', 'kite', 'tower', 'k', 'i', 'person', 'a', 'word', 'y', 'n'}
true node_word:kite
true node_word:person
gt_objects:{'branch', 'tree', 's', 'apparel', 'sky', 'l', 'g', 'shirt', 'r', '_', 'u', 'c', 't', 'e', 'building', 'jacket', 'field', 'b', 'd', 'string', 'awning', 'w', 'land', 'h', 'window', 'background', 'o', 'kite', 'tower', 'k', 'i', 'person', 'a', 'word', 'y', 'n'}
true node_word:kite
true node_word:person
gt_objects:{'branch', 'tree', 's', 'apparel', 'sky', 'l', 'g', 'shirt', 'r', '_', 'u', 'c', 't', 'e', 'building', 'jacket', 'field', 'b', 'd', 'string', 'awning', 'w', 'land', 'h', 'window', 'background', 'o', 'kite', 'tower', 'k', 'i', 'person', 'a', 'word', 'y', 'n'}
true node_word:kite
true node_word:person
gt_objects:{'wheel', 's', 'v', 'l', 'coat', 'bicycle', 'crack', 

 96%|█████████▌| 288/300 [00:06<00:00, 41.12it/s]

gt_objects:{'vase', 'tree', 'f', 's', 'chimney', 'v', 'bed', 'l', 'g', 'r', 'curtain', 'house', '_', 'c', 'u', 't', 'e', 'b', 'flower', 'pillow', 'd', 'q', 'w', 'h', 'window', 'blanket', 'm', 'o', 'blind', 'design', 'wall', 'k', 'frame', 'i', 'decoration', 'pane', 'a', 'p', 'fabric', 'n', 'shadow'}
true node_word:bed
gt_objects:{'vase', 'tree', 'f', 's', 'chimney', 'v', 'bed', 'l', 'g', 'r', 'curtain', 'house', '_', 'c', 'u', 't', 'e', 'b', 'flower', 'pillow', 'd', 'q', 'w', 'h', 'window', 'blanket', 'm', 'o', 'blind', 'design', 'wall', 'k', 'frame', 'i', 'decoration', 'pane', 'a', 'p', 'fabric', 'n', 'shadow'}
true node_word:window
negetive node_word:room
true node_word:bed
gt_objects:{'s', 'goggles', 'ski_pole', 'slope', 'l', 'g', 'letter', 'shirt', 'r', 'leg', 'hat', 'hand', 'c', '_', 'u', 't', 'e', 'snow', 'head', 'd', 'hood', 'skis', 'glove', 'face', 'h', 'foot', 'collar', 'trouser', 'm', 'o', 'k', 'i', 'person', 'a', 'p', 'y', 'n', 'shadow'}
true node_word:person
gt_objects:{'s',

100%|██████████| 300/300 [00:07<00:00, 41.22it/s]


true node_word:sink
negetive node_word:toilet
true node_word:bathroom
gt_objects:{'tree', 'lodger', 's', 'door', 'v', 'sky', 'l', 'g', 'vent', 'row', 'r', '_', 'c', 'u', 't', 'e', 'train', 'circle', 'car', 'h', 'pole', 'window', 'cloud', 'power_line', 'o', 'gravel', 'k', 'i', 'a', 'p', 'y', 'n'}
true node_word:train
negetive node_word:path
gt_objects:{'tree', 'lodger', 's', 'door', 'v', 'sky', 'l', 'g', 'vent', 'row', 'r', '_', 'c', 'u', 't', 'e', 'train', 'circle', 'car', 'h', 'pole', 'window', 'cloud', 'power_line', 'o', 'gravel', 'k', 'i', 'a', 'p', 'y', 'n'}
true node_word:train
negetive node_word:path
gt_objects:{'tree', 'lodger', 's', 'door', 'v', 'sky', 'l', 'g', 'vent', 'row', 'r', '_', 'c', 'u', 't', 'e', 'train', 'circle', 'car', 'h', 'pole', 'window', 'cloud', 'power_line', 'o', 'gravel', 'k', 'i', 'a', 'p', 'y', 'n'}
true node_word:train
gt_objects:{'d', 'f', 's', 'h', 'l', 'm', 'g', 'o', 'r', 'k', 'i', 'c', 'u', 'a', 'p', 't', 'e', 'n', 'b'}
negetive node_word:person
gt_o